In [1]:
try:
  %tensorflow_version 2.x
except:
  pass
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import pathlib
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import tensorflow_hub as hub

TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [4]:
from zipfile import ZipFile
file_name="/content/drive/My Drive/color.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
import os

In [0]:
classes=['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy','Blueberry___healthy','Cherry_(including_sour)___Powdery_mildew','Cherry_(including_sour)___healthy','Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_','Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy','Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)','Grape___healthy','Orange___Haunglongbing_(Citrus_greening)','Peach___Bacterial_spot','Peach___healthy','Pepper,_bell___Bacterial_spot','Pepper,_bell___healthy','Potato___Early_blight','Potato___Late_blight','Potato___healthy','Raspberry___healthy','Soybean___healthy','Squash___Powdery_mildew','Strawberry___Leaf_scorch','Strawberry___healthy','Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot','Tomato___Tomato_Yellow_Leaf_Curl_Virus','Tomato___Tomato_mosaic_virus','Tomato___healthy']

In [0]:
import glob
import shutil

In [8]:
project_path='/content/color'
for cl in classes:
  images = glob.glob('/content/color'+'/'+cl+'/*.JPG')
  print("{}: {} Images".format(cl, len(images)))
  train, val,test = images[:round(len(images)*0.7)], images[round(len(images)*0.78):round(len(images)*0.98)],images[round(len(images)*0.98):]
  
  for t in train:
    if not os.path.exists(os.path.join(project_path, 'train', cl)):
      os.makedirs(os.path.join(project_path, 'train', cl))
    shutil.move(t, os.path.join(project_path, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(project_path, 'val', cl)):
      os.makedirs(os.path.join(project_path, 'val', cl))
    shutil.move(v, os.path.join(project_path, 'val', cl))
  for t in test:
    if not os.path.exists(os.path.join(project_path, 'test', cl)):
      os.makedirs(os.path.join(project_path, 'test', cl))
    shutil.move(t, os.path.join(project_path, 'test', cl))

Apple___Apple_scab: 630 Images
Apple___Black_rot: 621 Images
Apple___Cedar_apple_rust: 275 Images
Apple___healthy: 1645 Images
Blueberry___healthy: 1502 Images
Cherry_(including_sour)___Powdery_mildew: 1052 Images
Cherry_(including_sour)___healthy: 854 Images
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 446 Images
Corn_(maize)___Common_rust_: 1192 Images
Corn_(maize)___Northern_Leaf_Blight: 870 Images
Corn_(maize)___healthy: 21 Images
Grape___Black_rot: 1180 Images
Grape___Esca_(Black_Measles): 1383 Images
Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1076 Images
Grape___healthy: 423 Images
Orange___Haunglongbing_(Citrus_greening): 5507 Images
Peach___Bacterial_spot: 2297 Images
Peach___healthy: 360 Images
Pepper,_bell___Bacterial_spot: 997 Images
Pepper,_bell___healthy: 1476 Images
Potato___Early_blight: 1000 Images
Potato___Late_blight: 1000 Images
Potato___healthy: 152 Images
Raspberry___healthy: 371 Images
Soybean___healthy: 5090 Images
Squash___Powdery_mildew: 1813 Images
Str

In [0]:

BATCH_SIZE=32

In [0]:
IMAGE_SHAPE = 224
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_SHAPE, IMAGE_SHAPE, 3))
feature_extractor.trainable=False


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
train_image_generator=ImageDataGenerator(rescale=1./255)
validation_image_generator=ImageDataGenerator(rescale=1./255)

In [0]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
                    )

In [14]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
        directory='/content/color/train',
        shuffle=True,
        target_size=(224,224),
        class_mode='binary')

Found 36961 images belonging to 38 classes.


In [15]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
        directory='/content/color/val',
        shuffle=False,
        target_size=(224,224),
        class_mode='binary')

Found 10563 images belonging to 38 classes.


In [17]:
test_image_generator=ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
        directory='/content/color/test',
        shuffle=False,
        target_size=(256,256),
        class_mode='binary')

Found 1055 images belonging to 37 classes.


In [0]:
model=tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(38,activation='softmax')
])

In [0]:
model.compile(
      optimizer='adam',
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

In [20]:
EPOCHS=20
history=model.fit_generator(
                   train_data_gen,
                   epochs=EPOCHS,
                   validation_data=val_data_gen)

Epoch 1/20
1156/1156 [==============================] - 469s 406ms/step - loss: 0.5633 - accuracy: 0.8476 - val_loss: 0.3305 - val_accuracy: 0.8958
Epoch 2/20
1156/1156 [==============================] - 459s 397ms/step - loss: 0.2538 - accuracy: 0.9218 - val_loss: 0.2354 - val_accuracy: 0.9244
Epoch 3/20
1156/1156 [==============================] - 457s 395ms/step - loss: 0.2114 - accuracy: 0.9319 - val_loss: 0.2542 - val_accuracy: 0.9156
Epoch 4/20
1156/1156 [==============================] - 458s 396ms/step - loss: 0.1876 - accuracy: 0.9394 - val_loss: 0.2415 - val_accuracy: 0.9211
Epoch 5/20
1156/1156 [==============================] - 457s 395ms/step - loss: 0.1728 - accuracy: 0.9440 - val_loss: 0.2062 - val_accuracy: 0.9330
Epoch 6/20
1156/1156 [==============================] - 460s 398ms/step - loss: 0.1645 - accuracy: 0.9458 - val_loss: 0.2113 - val_accuracy: 0.9318
Epoch 7/20
1156/1156 [==============================] - 469s 405ms/step - loss: 0.1556 - accuracy: 0.9482 - val_

In [0]:
RPS_SAVED_MODEL = "rps_saved_model"

In [0]:
tf.saved_model.save(model, RPS_SAVED_MODEL)

In [23]:
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 38)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [0]:
loaded = tf.saved_model.load(RPS_SAVED_MODEL)

In [25]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_input')})
{'dense': TensorSpec(shape=(None, 38), dtype=tf.float32, name='dense')}


In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(RPS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]


tflite_model = converter.convert()
with open("converted_model.tflite", "wb") as f:
  f.write(tflite_model)